In [1]:
import matplotlib.pyplot as plt
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, BatchNormalization
from keras.layers.pooling import GlobalAveragePooling1D
from keras.layers import Conv1D, MaxPooling1D, LSTM
from keras.layers.wrappers import TimeDistributed
from sklearn.metrics import confusion_matrix
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam
import numpy as np
import itertools
import glob
import os
import json
import sys
import time

Using Theano backend.


In [2]:
def get_model():
    model = Sequential([
        TimeDistributed(Conv1D(16,3, activation='relu', padding = "same"),input_shape=[45,18,2]),
        TimeDistributed(BatchNormalization()),
        #TimeDistributed(MaxPooling1D()),
        TimeDistributed(Dropout(0.5)),
        #TimeDistributed(Conv1D(64,3, activation='relu',padding = "same")),
        BatchNormalization(),
        #TimeDistributed(Dropout(0.8)),
        TimeDistributed(Flatten()),
        #TimeDistributed(Dense(30,activation='softmax')),  
        LSTM(20,unit_forget_bias = 0.5, return_sequences = True),
        TimeDistributed(Dense(6,activation='softmax'))        
    ])
    adam = Adam(lr=0.0001)
    model.compile(loss='categorical_crossentropy',
              optimizer= adam,
              metrics=['accuracy'])
    return model

In [3]:
asanas = {0:'bhujangasan', 1:'padamasan', 2:'shavasan', 3:'tadasan', 4:'trikonasan', 5:'vrikshasan'}

model = get_model()
model.load_weights("FINAL/weights/val1-73-0.9992.hdf5")

In [4]:
def addToPred(Xch,count,Xarr,Xlist,seq):
    Xarr[count] = Xch
    count = count + 1
    #print(count)
    if count == 45:
        #print('Hello2')
        Xtemp = []
        Xtemp.append(Xarr)
        Xlist[seq] = Xarr
        Xarr = np.empty((45,18,2))
        count = 0
        seq  = seq + 1
        #printing prediction for first frame        
        Yht_pred = model.predict(np.asarray(Xtemp))
        Y_pred = np.argmax(Yht_pred[0],axis = 1)
        Y_pred_asanas = np.array([asanas[a] for a in Y_pred])
        
        print("Prediction " + " is " + str(Y_pred_asanas[0]))
        
    #print(count)
    return seq, count , Xarr

In [5]:
def makePred(filename,count,Xarr,Xlist,seq):
    with open(filename) as json_data:
            d = json.load(json_data)
            for person in d['people']:
                A = []
                A = [person['pose_keypoints']]
                #A = [person['pose_keypoints_2d']]
                
                X = np.asarray(A)
                even = np.arange(0, X.shape[1], 3)
                odd = np.arange(1, X.shape[1], 3)
                X1 =  X[:,even]
                X2 = X[:,odd]
                Xch = np.dstack((X1,X2))
                seq ,count, Xarr = addToPred(Xch,count,Xarr,Xlist,seq)
                json_data.close
    return seq, count, Xarr

In [ ]:
count = 0
seq = 0
Xarr = np.empty((45,18,2))
Xlist = np.empty((1000,45,18,2))
#print(Xarr.shape)
lastRead = "none"
timeout = time.time() + 1

while(True):
    #print(count)
    list_of_files = glob.glob("output/*")
    #list_of_files = glob.glob("C:/Users/JLR/Desktop/yoga/openpose-windows/output/*")
    #latest_file = max(list_of_files, key=os.path.getmtime)
    latest_file = max(list_of_files, key=os.path.getctime)
    if lastRead != latest_file:
        try:
            seq, count, Xarr = makePred(latest_file,count,Xarr,Xlist,seq)
            timeout = time.time() + 5 # will give result if no new file for 5 secs
        except:
            print "Frame issue"
    if time.time() > timeout:
        break
    lastRead = latest_file

Prediction  is bhujangasan
